In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os



load_dotenv(".env")
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [12]:
from typing_extensions import Annotated, TypedDict, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from operator import add

class ConversationState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add]
    ai_character: str
    
def get_history(messages: Sequence[BaseMessage]) -> str:
    message = ""
    for msg in messages:
        message += f"\n{msg.content}"
    return message


character_prompt = PromptTemplate(
    input_variables=["ai_character", "history", "user_message"],
    template = """
    you are a {ai_character}. Respond to the user based on this persona.\n\n
    Conversation History: {history}\n\n
    User: {user_message}"""
)

def get_response_from_ai(state: ConversationState, user_messsage: str):
    if not state["ai_character"]:
        return "You have not selected a character."
    
    msg_history = get_history(state['messages'])
    prompt = character_prompt.format(
        ai_character=state["ai_character"],
        history=msg_history,
        user_message=user_messsage
    )
    
    response = openai.invoke(prompt)
    return response

init_state: ConversationState = {
    "messages": [],
    "ai_character": "french teacher"
}

input_msg = "Hi Teacher!"
response = get_response_from_ai(init_state, input_msg)
print(f"AI: {response}")
    

AI: 

    Teacher: Hello! How are you?


In [19]:
from typing_extensions import Annotated, TypedDict, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from operator import add

class ConversationState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add]
    ai_character: str
    
def get_history(messages: Sequence[BaseMessage]) -> str:
    message = ""
    for msg in messages:
        message += f"\n{msg.content}"
    return message


character_prompt = PromptTemplate(
    input_variables=["ai_character", "history", "user_message"],
    template = """
    you are a {ai_character}. Respond to the user based on this persona.\n\n
    Conversation History: {history}\n\n
    User: {user_message}"""
)

def get_response_from_ai(state: ConversationState, user_messsage: str):
    if not state["ai_character"]:
        return "You have not selected a character."
    
    msg_history = get_history(state['messages'])
    prompt = character_prompt.format(
        ai_character=state["ai_character"],
        history=msg_history,
        user_message=user_messsage
    )
    
    response = openai.invoke(prompt)
    return response

init_state: ConversationState = {
    "messages": [],
    "ai_character": "french teacher"
}

input_msg = "Hi Teacher!"
response = get_response_from_ai(init_state, input_msg)
print(f"AI: {response}")
    


state: ConversationState = {
    "messages": [],
    "ai_character": "french teacher"
}

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    
    user_msg = HumanMessage(content=user_input)
    state["messages"] += [user_msg]
    
    ai_res = get_response_from_ai(state, user_msg)
    assistant_message = AIMessage(content=ai_res)
    print(assistant_message)
    print(f"AI: {ai_res}")
    
    state["messages"] += [assistant_message]

for msg in state["messages"]:
    print(f"{msg.content}")

AI: 

Teacher: Hello! It's nice to meet you. I am your French teacher. How can I assist you?
content='\n\n\nTeacher: Hello! How can I assist you today?' additional_kwargs={} response_metadata={}
AI: 


Teacher: Hello! How can I assist you today?
content="\n\n    Teacher: No problem! We can start with some basic vocabulary and phrases. What's your name?" additional_kwargs={} response_metadata={}
AI: 

    Teacher: No problem! We can start with some basic vocabulary and phrases. What's your name?
content="\n\n    Teacher: We can begin by learning some basic greetings and introductions. Then we can move on to simple sentences and grammar rules. It's important to practice regularly and immerse yourself in the language as much as possible. Are there any particular areas you are interested in learning? " additional_kwargs={} response_metadata={}
AI: 

    Teacher: We can begin by learning some basic greetings and introductions. Then we can move on to simple sentences and grammar rules. It's 

In [23]:
from openai import OpenAI

openai = OpenAI()

with open("Recording.m4a", "rb") as record:
    transcription = openai.audio.transcriptions.create(
        model="whisper-1",
        file=record
    )
    print(transcription)

Transcription(text='Hey, my name is Abel.')


In [24]:
transcription.text

'Hey, my name is Abel.'

In [ ]:
import gradio as gr


character_one = """
You are a helpful assistant that teaches French language.
"""

character_two = """
You are the best comedian in the world. But you speak and crack jokes both in Nigerian Pidgin language and in English.
"""

MODEL = "gpt-4o-mini"
openai = OpenAI()

def chat_model(messages, stream=False):
    if stream:
        chunks = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=True
        )
        return chunks
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    return response

def chat(message, history, model_character):
    messages = []
    if model_character == "French Teacher":
        messages = [{"role": "system", "content": character_one}]
    elif model_character == "Comedian":
        messages = [{"role": "system", "content": character_two}]
    
    # Convert history to OpenAI format and append user message
    history = history or []
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        if assistant_message:
            messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})

    chunks = chat_model(messages, stream=True)
    responses = ""
    for chunk in chunks:
        delta_content = chunk.choices[0].delta.content or ""
        responses += delta_content
        if responses:  # Only yield if there’s content
            yield responses

def transcribe_audio(audio_file):
    if audio_file is None:
        return ""
    with open(audio_file, "rb") as audio:
        transcription = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio,
        )
    return transcription.text

# Custom interface with Blocks
with gr.Blocks() as b:
    gr.Markdown("# Choose a Preferred Character")
    character_dropdown = gr.Dropdown(
        choices=["French Teacher", "Comedian"],
        label="Select Character",
        value="French Teacher"
    )
    chatbot = gr.Chatbot(label="Kappa_ai")
    with gr.Row():
        textbox = gr.Textbox(label="Enter Text...")
        audio_mic = gr.Audio(source="microphone", type="filepath", label="Record Audio")
    btn = gr.Button("Send")
    
    def submit_message(message, history, character):
        if not message:
            return history or [], ""  # Return empty history and clear textbox if no message
        
        history = history or []
        history.append([message, None])
        for response in chat(message, history[:-1], character):
            history[-1][1] = response
            yield history, ""  # Yield updated history and clear textbox

    def submit_audio(audio, history, character):
        transcribed_text = transcribe_audio(audio)
        if not transcribed_text:
            return history or [], ""
        
        history = history or []
        history.append([transcribed_text, None])
        for response in chat(transcribed_text, history[:-1], character):
            history[-1][1] = response
            yield history, ""  # Clear textbox (optional, since audio-driven)

    textbox.submit(
        fn=submit_message,
        inputs=[textbox, chatbot, character_dropdown],
        outputs=[chatbot, textbox]
    )
    btn.click(
        fn=submit_audio,
        inputs=[audio_mic, chatbot, character_dropdown],
        outputs=[chatbot, textbox]  # Including textbox to keep outputs consistent
    )

b.queue().launch(inbrowser=True, share=True)

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://ab4df0412e9dd629be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


c:\Users\abelt\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
Traceback (most recent call last):
  File "c:\Users\abelt\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\processing_utils.py", line 145, in audio_from_file
    audio = AudioSegment.from_file(filename)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abelt\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\audio_segment.py", line 728, in from_file
    info = mediainfo_json(orig_file, read_ahead_limit=read_ahead_limit)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abelt\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py", line 274, in mediainfo_json
    res = Popen(command, stdin=stdin_parameter, stdo

In [27]:
b.close()

Closing server running on port: 7861


In [2]:
!ffmpeg -version

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.
